# Instrucciones.

1- Instalar requisitos previos: Python3 y Nginx
``sudo apt-get update``
``sudo apt install python3-pip nginx``

Nginx será nuestro servidor web, ¿Por qué nginx y no apache? Buena pregunta, no sé, ese ocupé nomás.

2- Subir el proyecto, dejémoslo en su carpetita y navegamos hasta ese directorio.

3- Setear entorno virtual para el proyecto
Estamos en la carpeta raíz del proyecto.
``pip3 install virtualenv``
``mkdir .env``
``python3 -m virtualenv .env``
Esperamos a que termine de hacer su coso y activamos el entorno
``source .env/bin/activate``
Debería aparecer (.env) antes de las órdenes de la consola, representando que estamos usando un entorno personalizado.

4- Instalamos django y gunicorn
``pip install django gunicorn``

5- Hacemos migraciones/migramos/colectamos static. Esto va a depender del proyecto, si no usamos base de datos las primeras dos son inútiles, si no usamos statics la última es inútil.

``python manage.py makemigrations``
``python manage.py migrate``
``python manage.py collectstatic``

como te darás cuenta, ``manage.py`` es la entrada a cualquiera wea que queramos hacer con el servidor de django. Puedes hacer ``python manage.py help`` para obtener la lista completa de comandos, incluso uno puede agregar comandos propios a esta wea.

Preocupémonos de que en ``settings.py`` tenemos una ruta para la variable ``STATIC_ROOT``, esa ruta es la carpeta que va a contener nuestros statics (css, javascript, fotos, etc). Para tener una ruta utilizando como referencia la raíz del proyecto, podemos asignar el valor ``os.path.join(BASE_DIR, 'static')``.

6- Ahora veamos si la cuestión arranca. Si esto funciona, estamos en fase de development y no nos sirve realmente para producción porque apenas cierres la consola, se va a caer.

``python manage.py runserver 0:8000``

Si no vemos errores, la raja, pero de seguro te va a saltar uno que habla de HOSTs no permitidos. Ahí mismo te dice cómo solucionarlo, agrega la IP del host a ``ALLOWED_HOSTS``en ``settings.py``. Si cachaste, es una variable de tipo list, así que puedes agregar varias strings con ip's de hosts. Es buena idea que pongas 'localhost' y '127.0.0.1' para ahorrarte quebradero de cabeza luego.

7- Listo, ya vimos que bonita mi página y la wea, ahora tenemos que hacer que sea un servicio del sistema, cosa que tan pronto como la VM se prenda, te levante el sitio.

Probemos levantar lo mismo, pero con Gunicorn haciendo 
``gunicorn --bind 0.0.0.0:8000 webpersonal.wsgi``

Al visitar deberíamos volver a ver el sitio web, pero ahora todo feo porque gunicorn no tiene puta idea de dónde están tus statics, no importa, se los vamos a servir con nginx.

vamos a ``/etc/systemd/system/`` y creamos un archivo ``gunicorn.service`` con el siguiente contenido:

    [Unit]
    Description=gunicorn daemon
    After=network.target
        
    [Service]
    User=maquinadjango
    Group=www-data
    WorkingDirectory=/home/maguinadjango/webpersonal/
    ExecStart=/home/maguinadjango/webpersonal/.env/bin/python3 /home/maguinadjango/webpersonal/.env/bin/gunicorn --workers 3 -b :8000 webpersonal.wsgi:application
        
    [Install]
    WantedBy=multi-user.target
      

  

Si te dai cuenta, lo único que estamos haciendo es correr el gunicorn igual que cuando lo hacíamos manual, pero ahora le estamos diciendo al OS que lo haga y estamos usando los ejecutables que están dentro del entorno virtual que hicimos.

Dale a comenzar el servicio ``sudo systemctl start gunicorn`` y enablea para que siempre se ejecute en arranque ``sudo systemctl enable gunicorn``, anda comprobando estado a ver si falló la wea usando ``sudo systemctl status gunicorn``

Pulento, ahora vamos al Nginx. 
En  ``/etc/nginx/sites-available/`` creamos un archivo ``webpersonal.conf`` con el siguiente contenido:

    server {
    
	    listen 80;
	    server_name 52.170.253.135;
	    client_max_body_size 50M;
	    
	    location = /favicon.ico { access_log off; log_not_found off; }
	    
	    location /static/ {
		    root /home/maquinadjango/webpersonal/static/;
	    }
	    
	    location / {
	    
		    proxy_pass http://127.0.0.1:8000/;
		    proxy_redirect off;
		    proxy_set_header X-Real-IP $remote_addr;
		    proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
		    proxy_connect_timeout 30;
		    proxy_read_timeout 30;
	    
	    }
    
    }

 Ya po, ahí cachai que estamos haciendo un montón de weas, pero las que destacan son asignar IP, puerto web, la ruta real a los static y que vamos a redireccionar desde el localhost (gunicorn).

Ahora linkeamos nuestra config de avaiable a los enabled haciendo: 
``sudo ln -s /etc/nginx/sites-available/webpersonal.conf /etc/nginx/sites-enabled`` y testeamos con ``sudo nginx -t`` , lloramos mientras corregimos la wea hasta que no tire errores el test, de seguro se pasó algún espacio weón o cualquier cosa, normalmente es la ruta de los static.


En este punto, si todo está funcionando, pulento, en mi caso lloro porque no me está encontrando los static, pero siempre son puras weas... Busca el error con ``
sudo tail -30 /var/log/nginx/error.log`` y cacha que la ruta de los static sea correcta, sino repara. Yo creo que traté un montón de weas y puede que el final de los archivos no sea el que especifiqué en un principio xd, pero igual no debería ser difícil mirar los archivos y cachar las diferencias. Bueno eso chau.